In [1]:
from pyspark.sql import SparkSession
from scipy import sparse
import numpy as np
import pandas as pd
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from  pyspark.sql.functions import isnan
# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert a Pandas DF to a Spark DF
#3spark_df = spark.createDataFrame(pandas_df) 

# Convert a Spark DF to a Pandas DF
#pandas_df = spark_df.toPandas()

In [2]:
df = pd.read_csv('../data/training.csv')
df=df.sort_values(by=['timestamp'], ascending=False)


In [3]:
#df.describe(percentiles=[0.8])

In [4]:
df_test=df[df['timestamp']>974737385]
df_train=df[df['timestamp']<=974737385]

In [5]:
ratings_df_train = spark.createDataFrame(df_train)
ratings_df_test= spark.createDataFrame(df_test)

In [6]:
als = ALS(
    itemCol='movie',
    userCol='user',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.1,
    rank=18)

In [7]:
recommender_train = als.fit(ratings_df_train)

In [8]:
train_recs = recommender_train.transform(ratings_df_train)

In [9]:
# train_recs.select(isnan('prediction')).groupby('isnan(prediction)').count().show()

In [10]:
#pandas_df = train_recs.toPandas()

In [11]:
#pandas_df.describe()

In [12]:
#pandas_df.rating.hist(bins=5,density=True)
#plt.show()

In [13]:
#pandas_df.prediction.hist(bins=5,density=True)

In [14]:
test_ratings_df = spark.createDataFrame(df_test)

In [16]:
test_recs = recommender_train.transform(test_ratings_df)

In [17]:
#test_recs.select(isnan('prediction')).groupby('isnan(prediction)').count().show()

In [18]:
submission_cross_val = test_recs.toPandas()

In [19]:
#submission_cross_val.rename(columns={'prediction': 'rating'}, inplace=True)


In [20]:
#submission_cross_val.to_csv('../data/mvp_submission_cv.csv',index=False)

In [21]:
submission_cross_val.head()

,user,movie,rating,timestamp,prediction
0,673,148,5,975620824,NaN
1,1242,148,3,974909976,NaN
2,1069,148,2,974945135,NaN
3,1605,148,2,974930221,1.940744
4,1150,148,2,974875106,NaN


In [22]:
#submission_cross_val['prediction'].fillna(df_train['rating'].median(),inplace=True)

In [23]:
submission_cross_val['prediction'].fillna(df_train['rating'].mean(),inplace=True)

In [24]:



g = submission_cross_val.groupby('user')
top_5 = g.prediction.transform(lambda x: x >= x.quantile(.95))
       

submission_cross_val['rating'][top_5==True].mean()

3.5448300514430473

In [ ]:


g = submission_cross_val.groupby('user')
top_5 = g.prediction.transform(lambda x: x >= x.quantile(.95))
       

submission_cross_val['rating'][top_5==True].mean()

In [ ]:
top_5.head()

In [ ]:
gr = submission_cross_val.groupby('user')
top_5r = gr.rating.transform(lambda x: x >= x.quantile(.95))
       

submission_cross_val['rating'][top_5r==1].mean()

In [ ]:
df_movies = pd.read_csv('../data/movies.dat', sep = '::', header = None, skiprows = 1) 
df_users = pd.read_csv('../data/users.dat', sep = '::', header = None, skiprows = 1) 

In [ ]:
g = sub_mean.groupby('user')
top_5 = g.prediction.transform(lambda x: x >= x.quantile(.95))
       

submission_cross_val['rating'][top_5==1].mean()